In [1]:
# import urllib.request
# api_key = ''
# crypto = 'DOGE'
# interval = '1d'
# # url = "https://api.nomics.com/v1/currencies/ticker?key=" + api_key + \
# #       "&ids=" + crypto + "&interval=" + interval + "&convert=USD&per-page=100&page=1"

# start_date = '2020-01-01'
# end_date = '2020-12-31'
# url = "https://api.nomics.com/v1/currencies/sparkline?key=" + api_key + \
#       "&ids=" + crypto + "&start=" + start_date + "T00%3A00%3A00Z&end=" + end_date + "T00%3A00%3A00Z" + "&interval=" + interval

# print(urllib.request.urlopen(url).read())

In [4]:
# Import the libraries
import numpy as np
import pandas as pd

In [5]:
# Load the data
df = pd.read_csv('doge.csv')

# Set date as the index 
df.set_index(pd.DatetimeIndex(df['date']), inplace=True)

# Remove unwanted data
df = df[['PriceUSD']]
df = df.iloc[2300:,:]

# Show the data
df.head()

,PriceUSD
date,
2020-05-11,0.002406
2020-05-12,0.002433
2020-05-13,0.002564
2020-05-14,0.002587
2020-05-15,0.002482


In [6]:
# Create a variable to store the number of days into the future that we want to predict
prediction_days = 1

# Create a new column called Prediction
df['Prediction'] = df['PriceUSD'].shift(-prediction_days)

# Show the data
df

,PriceUSD,Prediction
date,,
2020-05-11,0.002406,0.002433
2020-05-12,0.002433,0.002564
2020-05-13,0.002564,0.002587
2020-05-14,0.002587,0.002482
2020-05-15,0.002482,0.002470
...,...,...
2021-06-22,0.188829,0.234789
2021-06-23,0.234789,0.263518
2021-06-24,0.263518,0.240247


In [7]:
# Create the independent dataset (X)
X = np.array(df.drop(['Prediction'], 1))

# Remove the last n+1 rows of data where n is the prediction_days
X = X[: len(df) - prediction_days - 1]

# Show the data
X[-10:]

array([[0.3206744 ],
       [0.30715372],
       [0.30584986],
       [0.29246527],
       [0.28729606],
       [0.28066914],
       [0.17918748],
       [0.18882902],
       [0.23478921],
       [0.26351766]])

In [8]:
# Create a dependent dataset(y)
y = np.array(df['Prediction'])

# Get all of the y values except the last n+1 rows
y = y[: -prediction_days-1]

# Show the data
y[-10:]

array([0.30715372, 0.30584986, 0.29246527, 0.28729606, 0.28066914,
       0.17918748, 0.18882902, 0.23478921, 0.26351766, 0.24024652])

In [9]:
# !pip install sklearn

In [10]:
# Split the data into train and test dataset
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)  

In [11]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators = 2, random_state = 587)
forest.fit(x_train, y_train)
print(forest.score(x_test, y_test))

0.9748421826205664


In [12]:
# Show how close the predicted values and the actual values are
predicted = forest.predict(x_test)

print(predicted)
print()

print(y_test)

[0.05798004 0.00260422 0.0031786  0.00261899 0.00252266 0.00254786
 0.05623937 0.27658023 0.04956684 0.00249849 0.00249947 0.57398368
 0.05648384 0.00254786 0.00353665 0.04883477 0.00284449 0.00232932
 0.00285134 0.05405095 0.07090206 0.00269316 0.35450384 0.00249947
 0.00254698 0.47266653 0.00261142 0.32229974 0.00252933 0.00345465
 0.0025947  0.00292778 0.00352669 0.27171032 0.05527366 0.05798004
 0.00332409 0.05684285 0.00277759 0.00249947 0.00373675 0.00285134
 0.05736062 0.00810836 0.00312569 0.00912897 0.00747906 0.05798004
 0.00450374 0.00948991 0.50883898 0.0032733  0.57398368 0.27171032
 0.00353817 0.00341584 0.00254698 0.00353817 0.00323495 0.00246923
 0.32356871 0.00338891 0.00269825 0.05755861 0.00341584 0.00232932
 0.0042216  0.00232932 0.50883898 0.00323304 0.00289144 0.00359071
 0.00257289 0.00856188 0.00269316 0.00359517 0.0027926  0.00276499
 0.30949183 0.05755861 0.00319713 0.00254106]

[0.0571479  0.00259199 0.0032117  0.00265016 0.00246756 0.00251088
 0.05742943 0.1

In [13]:
# Get the validation data of the model
# Create a variable to store all of the rows in the data set except the last n rows
temp_df = df[: -prediction_days]

# Create a variable to store the independent price value
x_value = temp_df.tail(1)['PriceUSD'][0]

print(x_value)

0.240246524


In [14]:
prediction = forest.predict([[x_value]])

print('The price of Dogecoin in', prediction_days, 'day(s) is predicted to be', prediction[0])

print('The actual price was', temp_df.tail(1)['Prediction'][0])

The price of Dogecoin in 1 day(s) is predicted to be 0.271710321
The actual price was 0.24094331
